In [2]:
!pip install fastapi uvicorn kafka-python

  Using cached fastapi-0.95.0-py3-none-any.whl (57 kB)
  Using cached uvicorn-0.21.1-py3-none-any.whl (57 kB)
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
  Using cached starlette-0.26.1-py3-none-any.whl (66 kB)
  Using cached pydantic-1.10.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [ ]:
from fastapi import FastAPI
from kafka import KafkaProducer
import json
import os
import logging
import cProfile

logger = logging.getLogger(__name__)  # the __name__ resolve to "uicheckapp.services"

# this is a hack to multiply the work-load inserted into Kafka
names_to_multiply = os.environ.get('NAMES_TO_MULTIPLY', '').split(",")

_producer = None
try:
    _producer = KafkaProducer(bootstrap_servers=os.environ.get('KAFKA_BROKER', 'broker1:9093').split(","), 
                              api_version=(0, 10),
                              max_block_ms=10000)
except Exception as ex:
    logger.error('Exception while connecting Kafka')
    logger.error(str(ex))


app = FastAPI()


@app.get("/")
async def root():
    print("mega sus")
    return {"message": "Hello World"}


# insert URL into SensorLogger app: http://100.102.3.111:8080/data?device=Pixel6&person=Florin&activity=running
# https://github.com/tszheichoi/awesome-sensor-logger/#live-data-streaming
@app.post("/data")
async def data(data: dict, activity: str = None, device: str = None, person: str = None):
    profiler.enable()
    try:
        for measurement in data["payload"]:
            timestamp_ms = int(measurement["time"]) // 1_000_000 # convert nanoseconds to milliseconds
            value_bytes = bytes(json.dumps(measurement["values"]), encoding='utf-8')
            for name in names_to_multiply + [person]:
                key_bytes = bytes(f"{name}:{device}:{activity}", encoding='utf-8')
                _producer.send(measurement['name'],
                            key=key_bytes,
                            value=value_bytes,
                            timestamp_ms=timestamp_ms) 
        #_producer.flush()
        logger.info('Message published successfully.')
    except Exception as ex:
        logger.error('Exception in publishing message')
        logger.error(str(ex))
    profiler.disable()
    raise Exception
    return {"message": "Sensor data received"}

import uvicorn
import asyncio
import nest_asyncio

profiler = cProfile.Profile()


if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)
    """config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.create_task(server.serve())
"""

INFO:     Started server process [989]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
